# imports

In [4]:
import paho.mqtt.client as mqtt
import mysql.connector
from datetime import datetime, timedelta
import json
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# MySQL Configuration and Database Initialization

Once the connection to the MQTT server is successfully established, two key functions are executed:

 - create_database_if_not_exists(cursor):
        This function checks whether the GaiaBase database exists. If it doesn't, the database is created.
        If the creation fails, an error message is printed, and the program exits.

 - create_table_if_not_exists(cursor):
        After ensuring the database is in use, this function checks if the sensor_data2 table exists.
        If the table is missing, it's created with columns like soil_moisture, temperature, humidity, lat, lng, and a timestamp.
        Any errors during table creation are handled similarly with an error message and program termination.

In [5]:
config = {
    'user': 'GaiaDatabaseUser',
    'password': 'brahimAZE123',
    'host': 'localhost',
    'database': 'GaiaBase'
}


# these functions are ran when MQTT server connects successfully
def create_database_if_not_exists(cursor):
    try:
        cursor.execute("CREATE DATABASE IF NOT EXISTS GaiaBase")
        print("Database 'GaiaBase' checked/created.")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")
        exit(1)

def create_table_if_not_exists(cursor):
    cursor.execute("USE GaiaBase")
    table_creation_query = """
    CREATE TABLE IF NOT EXISTS sensor_data2 (
        id INT AUTO_INCREMENT PRIMARY KEY,
        soil_moisture INT NOT NULL,
        temperature FLOAT NOT NULL,
        humidity FLOAT NOT NULL,
        lat FLOAT NOT NULL,
        lng FLOAT NOT NULL,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    try:
        cursor.execute(table_creation_query)
        print("Table 'sensor_data' checked/created.")
    except mysql.connector.Error as err:
        print(f"Failed creating table: {err}")
        exit(1)

Inserting JSON Data into MySQL

The insert_json_data(json_data) function is designed to parse and insert sensor data from a JSON object into the MySQL sensor_data2 table. Here's a breakdown of its key steps:

- Connect to MySQL: A connection to the MySQL server is established using the configuration dictionary config, which contains details like the database user, password, host, and database name.

- Parse JSON Data: The function parses the provided JSON data using the json.loads() method to extract sensor values such as soil_moisture, temperature, humidity, lat, and lng. These values correspond to the fields in the sensor_data2 table.

- Insert Data: Using an SQL INSERT query, the parsed sensor data is inserted into the sensor_data2 table. The query uses placeholders (%s) to safely bind the values and execute the statement.

- Error Handling: If any errors occur during the database connection or data insertion (e.g., invalid data or database issues), an error message is printed.

- Close Connection: After the operation, the cursor and the database connection are closed to ensure resources are freed.

In [7]:
def insert_json_data(json_data):
    try:
        # Connect to MySQL server
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()


        # Parse JSON data
        data = json.loads(json_data)
        soil_moisture = data.get('soil_moisture')
        temperature = data.get('temperature')
        humidity = data.get('humidity')
        lat = data.get('lat')
        lng = data.get('lng')

        # Insert data into table
        insert_query = """
        INSERT INTO sensor_data2 (soil_moisture, temperature, humidity, lat, lng)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (soil_moisture, temperature, humidity, lat, lng))
        conn.commit()
        print("Data inserted successfully.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        conn.close()

# MQTT Client Setup for Data Handling

The code sets up an MQTT client to receive sensor data and dispatch it to the appropriate callback functions for further processing. Here's a breakdown of the key steps:

- Create MQTT Client: An instance of the MQTT client is created using mqtt.Client(), which will handle the connection to the broker and manage the communication.

- Set Callback Functions: The client is configured with two key callback functions:
        [on_connect]: This function is triggered when the client successfully connects to the MQTT broker. Typically, it subscribes to topics or confirms the connection.
        [on_message]: This function is called whenever a message is received on a subscribed topic, handling incoming sensor data (likely JSON) that will be inserted into the database.

- Connect to the MQTT Broker: The client connects to the MQTT broker running on the local server (localhost) at port 1883 (default for MQTT).

- Start the Event Loop: The client.loop_forever() method starts the loop that processes network traffic.

In [8]:
# Define the callback function for when a message is published
def on_publish(client, userdata, mid):
    print("Message published with mid: {}".format(mid))


# Define the callback function for when a message is received
def on_message(client, userdata, message):
    insert_json_data(str(message.payload.decode("utf-8")))
    print(f"Topic: {message.topic}\nMessage: {message.payload.decode()}")


# Define the callback function for when the client connects to the broker
def on_connect(client, userdata, flags, rc):
    # Connect to MySQL server
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # Create database and table if not exists
    create_database_if_not_exists(cursor)
    create_table_if_not_exists(cursor)

    print(f"Connected with result code {rc}")
    client.subscribe("gaia-lora-receiver/outgoing")


    cursor.close()
    conn.close()

In [9]:
# Create an MQTT client instance
client = mqtt.Client()

# Set the callback functions
client.on_connect = on_connect
client.on_message = on_message
client.on_publish = on_publish

# Connect to the MQTT broker running on the same server
client.connect("localhost", 1883, 60)

/tmp/ipykernel_4925/1575112037.py:2: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

## Test the connectivity between server and MQTT

In [10]:
# Publish the message
topic = "gaia-lora-receiver/incoming"
result = client.publish(topic, "hello from the claud")

# Check if the publish was successful
if result.rc == mqtt.MQTT_ERR_SUCCESS:
    print("Message sent successfully.")
else:
    print("Failed to send message.")

# Loop to process network traffic and callbacks
client.loop_start()

# Optional: wait for a moment before disconnecting
import time
time.sleep(5)

# Disconnect from the broker
client.loop_stop()
client.disconnect()

Message published with mid: 1
Message sent successfully.
Database 'GaiaBase' checked/created.
Table 'sensor_data' checked/created.
Connected with result code 0


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

# Irrgation system

To determine if irrigation is needed and for how long based on the data of the past day, you can analyze the soil moisture, temperature, and humidity values in the context of the past day's readings from your MySQL database.

Assumptions:

    Soil Moisture: If the soil moisture is below 30%, irrigation is needed.
    Temperature: Higher temperatures can accelerate water evaporation from the soil, so you can adjust irrigation time based on the temperature.
    Humidity: Low humidity can also increase water evaporation, leading to more irrigation needs.

Explanation:

    Query Data: It queries the past 24 hours of data from the sensor_data table.
    Irrigation Calculation:
        It checks whether the soil moisture is below the threshold (30% in this case).
        It adjusts the irrigation time based on the temperature (higher temperatures increase the irrigation time) and humidity (lower humidity increases the irrigation time).

Output: If irrigation is needed, it outputs the total time (in seconds); otherwise, it says no irrigation is needed.

### Fetching data

In [12]:
# Connect to the MySQL database
connection = mysql.connector.connect(**config)
cursor = connection.cursor()

# Calculate the timestamp for 24 hours ago
one_day_ago = datetime.now() - timedelta(days=1)

# Query to fetch data from the past 24 hours
query = """
SELECT soil_moisture, temperature, humidity, timestamp 
FROM sensor_data2 
WHERE timestamp >= %s
"""
cursor.execute(query, (one_day_ago,))

# Fetch all results from the executed query
results = cursor.fetchall()

# Convert the results into a pandas DataFrame
# Define the column names that correspond to the SELECT query
df = pd.DataFrame(results, columns=["soil_moisture", "temperature", "humidity", "timestamp"])

# Display the DataFrame
print(df)

# Close the cursor and connection
cursor.close()
connection.close()

Empty DataFrame
Columns: [soil_moisture, temperature, humidity, timestamp]
Index: []


### Define thresholds and base irrigation time

In [ ]:
moisture_threshold = 30  # Moisture percentage threshold
base_irrigation_time = 600  # Base irrigation time in seconds (10 minutes)
irrigation_time = 0

for row in results:
    soil_moisture, temperature, humidity, timestamp = row
    
    # Check if soil moisture is below the threshold
    if soil_moisture < moisture_threshold:
        # Calculate irrigation time adjustments based on temperature and humidity
        temp_factor = 1 + (temperature - 25) * 0.05  # Adjust based on temperature
        humidity_factor = 1 - (humidity / 100)  # Adjust based on humidity
        irrigation_time += base_irrigation_time * temp_factor * humidity_factor

# If irrigation is needed, return the time in seconds
if irrigation_time > 0:
    print(f"Irrigation needed for {irrigation_time:.2f} seconds.")
    lat, lng = get_last_lat_and_lng()
    irrigation_decision = weather(lat, lng)
    if (irrigation_decision):
        sendduration(irrigation_time*1000)
else:
    print("No irrigation needed.")

### running the irrigation system

In [ ]:
while(True):
    time.sleep(1000*60*60*3)
    # 3 hour delay
    check_irrigation()

# Weather

In [15]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

def should_irrigate(hourly_dataframe):
    # Define thresholds
    high_precipitation_probability = 50  # percent
    significant_rain = 5  # mm
    high_temperature = 30  # Celsius (soil or air temperature)
    
    # Look at the next few hours of weather data (e.g., next 6 hours)
    next_hours = hourly_dataframe.head(6)

    # Check for rain and precipitation probability
    avg_precipitation_probability = next_hours["precipitation_probability"].mean()
    total_rain = next_hours["rain"].sum()

    # Check for high temperatures that might require irrigation
    avg_soil_temperature_0cm = next_hours["soil_temperature_0cm"].mean()

    # Decision based on weather conditions
    if total_rain > significant_rain:
        print("No irrigation needed: Significant rain forecasted.")
        return False
    elif avg_precipitation_probability > high_precipitation_probability:
        print("No irrigation needed: High chance of precipitation.")
        return False
    elif avg_soil_temperature_0cm > high_temperature:
        print("Irrigation needed: High soil temperature detected.")
        return True
    else:
        print("Irrigation needed: Low rain and precipitation probability.")
        return True

def weather(lat, lng):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Make sure all required weather variables are listed here
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lng,
        "hourly": [
            "temperature_2m", 
            "relative_humidity_2m", 
            "precipitation_probability", 
            "rain", 
            "soil_temperature_0cm", 
            "soil_temperature_6cm", 
            "soil_temperature_18cm"
        ],
        "past_days": 7
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
        "precipitation_probability": hourly.Variables(2).ValuesAsNumpy(),
        "rain": hourly.Variables(3).ValuesAsNumpy(),
        "soil_temperature_0cm": hourly.Variables(4).ValuesAsNumpy(),
        "soil_temperature_6cm": hourly.Variables(5).ValuesAsNumpy(),
        "soil_temperature_18cm": hourly.Variables(6).ValuesAsNumpy(),
    }

    hourly_dataframe = pd.DataFrame(data=hourly_data)
    print(hourly_dataframe)

    # Determine if irrigation is needed based on weather data
    irrigation_needed = should_irrigate(hourly_dataframe)
    return irrigation_needed


How This Works:

- Data Collection: The weather forecast is retrieved using Open-Meteo, and the data is processed into a pandas.DataFrame for easier manipulation.

- Decision Logic: The function should_irrigate() checks the weather data for the next 6 hours:
            If the average precipitation probability is above 50%, irrigation is not needed.
            If more than 5mm of rain is expected, irrigation is skipped.
            If soil temperature at 0cm depth exceeds 30°C, irrigation is recommended to compensate for likely evaporation.

- Output: The system will print the decision and return True or False based on whether irrigation is required.